# Day 11: Decision Trees - Interpretable Models with Sharp Edges

**MGMT 47400 - Predictive Analytics**  
**4-Week Online Course**  
**Day 11 - Tuesday June 1, 2027**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davi-moreira/2026Summer_predictive_analytics_purdue_MGMT474/blob/main/notebooks/11_decision_trees.ipynb)

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Fit decision trees for regression/classification
2. Control complexity (depth, min samples) to manage overfitting
3. Interpret tree structure and failure modes
4. Compare tree vs linear/logistic baselines under CV
5. Document "when a tree is the right tool"

---

In [ ]:
# Setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer, fetch_california_housing
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.precision', 4)
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
print("✓ Setup complete!")

## 1. Decision Tree Intuition

### How Trees Make Decisions

**Algorithm (CART - Classification and Regression Trees):**
1. Start with all data at root
2. Find best feature + threshold to split
   - "Best" = maximize information gain (classification) or minimize MSE (regression)
3. Create two child nodes
4. Repeat recursively until stopping criteria

**Example Decision Path:**
```
Is income > $50k?
  ├─ No: Is age > 30?
  │   ├─ No: Predict class 0 (high risk)
  │   └─ Yes: Predict class 1 (low risk)
  └─ Yes: Predict class 1 (low risk)
```

### Key Hyperparameters

**Complexity control:**
- `max_depth`: Maximum tree depth (prevents overfitting)
- `min_samples_split`: Minimum samples to split a node
- `min_samples_leaf`: Minimum samples in leaf node
- `max_features`: Number of features to consider per split

## 2. Classification Tree Example

In [ ]:
# Load data
data = load_breast_cancer(as_frame=True)
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED, stratify=y)

print(f"Train: {len(X_train)} | Test: {len(X_test)}")
print(f"Features: {X.shape[1]}")

# Fit a simple tree
tree_clf = DecisionTreeClassifier(max_depth=3, random_state=RANDOM_SEED)
tree_clf.fit(X_train, y_train)

train_score = tree_clf.score(X_train, y_train)
test_score = tree_clf.score(X_test, y_test)

print(f"\n=== DECISION TREE (max_depth=3) ===")
print(f"Train accuracy: {train_score:.4f}")
print(f"Test accuracy: {test_score:.4f}")
print(f"Overfit gap: {train_score - test_score:.4f}")

In [ ]:
# Visualize the tree
plt.figure(figsize=(20, 10))
plot_tree(
    tree_clf,
    feature_names=X.columns,
    class_names=data.target_names,
    filled=True,
    rounded=True,
    fontsize=10
)
plt.title("Decision Tree Visualization (max_depth=3)")
plt.tight_layout()
plt.show()

print("💡 Follow a path from root to leaf to see decision logic")
print("💡 Darker colors = more samples, purity of class")

## 3. The Overfitting Problem

### Trees Without Constraints = Memorization

In [ ]:
# Compare different depths
depths = [1, 2, 3, 5, 10, 20, None]  # None = unlimited
results = []

for depth in depths:
    tree = DecisionTreeClassifier(max_depth=depth, random_state=RANDOM_SEED)
    tree.fit(X_train, y_train)
    
    train_acc = tree.score(X_train, y_train)
    test_acc = tree.score(X_test, y_test)
    
    results.append({
        'max_depth': str(depth),
        'train_acc': train_acc,
        'test_acc': test_acc,
        'gap': train_acc - test_acc,
        'n_leaves': tree.get_n_leaves()
    })

results_df = pd.DataFrame(results)
print("=== DEPTH SWEEP ===")
print(results_df.to_string(index=False))

# Plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy vs depth
axes[0].plot(range(len(depths)), results_df['train_acc'], marker='o', label='Train', linewidth=2)
axes[0].plot(range(len(depths)), results_df['test_acc'], marker='s', label='Test', linewidth=2)
axes[0].set_xticks(range(len(depths)))
axes[0].set_xticklabels(results_df['max_depth'])
axes[0].set_xlabel('Max Depth')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Accuracy vs Tree Depth')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Overfit gap
axes[1].bar(range(len(depths)), results_df['gap'], alpha=0.7, edgecolor='black')
axes[1].set_xticks(range(len(depths)))
axes[1].set_xticklabels(results_df['max_depth'])
axes[1].set_xlabel('Max Depth')
axes[1].set_ylabel('Train - Test Gap')
axes[1].set_title('Overfitting vs Tree Depth')
axes[1].axhline(y=0, color='r', linestyle='--')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Unrestricted trees achieve 100% training accuracy (pure overfitting!)")
print("💡 Best test performance is at moderate depth")

## 📝 PAUSE-AND-DO Exercise 1 (10 minutes)

**Task:** Run a depth sweep and choose depth based on CV.

---

In [ ]:
# YOUR CODE: Depth sweep with cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
depths_to_test = [2, 3, 4, 5, 6, 7, 8, 10, 15]

cv_results = []
for depth in depths_to_test:
    tree = DecisionTreeClassifier(max_depth=depth, random_state=RANDOM_SEED)
    scores = cross_val_score(tree, X_train, y_train, cv=cv, scoring='roc_auc')
    
    cv_results.append({
        'max_depth': depth,
        'cv_mean': scores.mean(),
        'cv_std': scores.std()
    })

cv_df = pd.DataFrame(cv_results)
print("=== CV DEPTH SWEEP ===")
print(cv_df.to_string(index=False))

best_depth = cv_df.loc[cv_df['cv_mean'].idxmax(), 'max_depth']
best_score = cv_df.loc[cv_df['cv_mean'].idxmax(), 'cv_mean']

print(f"\n✓ Best depth by CV: {best_depth} (CV ROC-AUC = {best_score:.4f})")

# Plot
plt.figure(figsize=(10, 6))
plt.errorbar(cv_df['max_depth'], cv_df['cv_mean'], yerr=cv_df['cv_std'], 
             marker='o', capsize=5, linewidth=2)
plt.axvline(x=best_depth, color='r', linestyle='--', label=f'Best depth = {best_depth}')
plt.xlabel('Max Depth')
plt.ylabel('CV ROC-AUC')
plt.title('Cross-Validation: Depth Selection')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. Tree vs Linear Model Comparison

In [ ]:
# Compare tree vs logistic regression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

models = {
    'Decision Tree (best)': DecisionTreeClassifier(max_depth=int(best_depth), random_state=RANDOM_SEED),
    'Logistic Regression': Pipeline([
        ('scaler', StandardScaler()),
        ('clf', LogisticRegression(random_state=RANDOM_SEED, max_iter=1000))
    ])
}

comparison = []
for name, model in models.items():
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='roc_auc')
    
    # Fit on full train for test evaluation
    model.fit(X_train, y_train)
    test_score = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    comparison.append({
        'Model': name,
        'CV_Mean': scores.mean(),
        'CV_Std': scores.std(),
        'Test_Score': test_score
    })

comp_df = pd.DataFrame(comparison)
print("=== TREE VS LINEAR COMPARISON ===")
print(comp_df.to_string(index=False))

print("\n💡 Which performs better depends on data structure")
print("💡 Trees handle non-linear relationships naturally")
print("💡 Linear models need manual feature engineering")

## 📝 PAUSE-AND-DO Exercise 2 (10 minutes)

**Task:** Write 3 observed tree failure modes (with evidence).

---

### YOUR ANALYSIS: Tree Failure Modes

**Failure Mode 1: Overfitting**  
[Evidence from depth sweep - what happened with unlimited depth?]

**Failure Mode 2: Instability**  
[Evidence from CV std - how much do scores vary across folds?]

**Failure Mode 3: Extrapolation**  
[Trees can only predict values seen in training - what's the implication?]

---

## 5. Regression Trees

In [ ]:
# Load regression dataset
california = fetch_california_housing(as_frame=True)
X_reg = california.data
y_reg = california.target

X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=RANDOM_SEED
)

# Depth sweep for regression
depths_reg = [2, 3, 5, 7, 10, 15, None]
reg_results = []

for depth in depths_reg:
    tree_reg = DecisionTreeRegressor(max_depth=depth, random_state=RANDOM_SEED)
    tree_reg.fit(X_train_reg, y_train_reg)
    
    train_r2 = tree_reg.score(X_train_reg, y_train_reg)
    test_r2 = tree_reg.score(X_test_reg, y_test_reg)
    
    reg_results.append({
        'max_depth': str(depth),
        'train_r2': train_r2,
        'test_r2': test_r2,
        'gap': train_r2 - test_r2
    })

reg_df = pd.DataFrame(reg_results)
print("=== REGRESSION TREE DEPTH SWEEP ===")
print(reg_df.to_string(index=False))

print("\n💡 Same overfitting pattern as classification")
print("💡 Unrestricted tree memorizes training data")

## 6. When to Use Decision Trees

### Strengths
- ✓ Interpretable (can visualize and explain)
- ✓ Handle non-linear relationships naturally
- ✓ No feature scaling needed
- ✓ Handle mixed data types (numeric + categorical)
- ✓ Capture interactions automatically
- ✓ Fast to train and predict

### Weaknesses
- ✗ High variance (unstable - small data changes → different tree)
- ✗ Easy to overfit
- ✗ Poor extrapolation (can't predict outside training range)
- ✗ Biased toward features with many values
- ✗ Step-function boundaries (not smooth)

### Use Decision Trees When:
1. Interpretability is critical
2. You need quick baseline
3. Relationships are highly non-linear
4. You'll use ensemble methods (Random Forests, Boosting)

### Avoid Decision Trees When:
1. High-dimensional sparse data
2. Need smooth decision boundaries
3. Small datasets (unstable)
4. Extrapolation required

## 7. Wrap-Up: Key Takeaways

### What We Learned Today:

1. **Tree Mechanics**: Recursive partitioning with greedy splits
2. **Overfitting Risk**: Unrestricted trees memorize perfectly
3. **Complexity Control**: depth, min_samples_split, min_samples_leaf
4. **Interpretability**: Can visualize exact decision logic
5. **Limitations**: High variance, poor extrapolation

### Critical Rules:

> **"Never use unrestricted trees in production"**

> **"Always tune depth with cross-validation"**

> **"Trees are building blocks for ensembles"**

### Next Steps:

- Day 12: Random Forests (ensemble of trees)
- We'll fix tree instability with bagging
- Learn feature importance from forests

---

## Bibliography

- James, G., Witten, D., Hastie, T., & Tibshirani, R. (2021). *An Introduction to Statistical Learning with Python* - Tree-Based Methods (trees, pruning)
- Hastie, T., Tibshirani, R., & Friedman, J. (2009). *The Elements of Statistical Learning* - CART foundations and complexity control
- Breiman, L., Friedman, J., Stone, C. J., & Olshen, R. A. (1984). *Classification and Regression Trees*
- scikit-learn User Guide: [Decision Trees](https://scikit-learn.org/stable/modules/tree.html)

---

**End of Day 11 Notebook**